In [5]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact,interactive
import ipywidgets as widgets



In [6]:
def get_closest_point(clic_x,clic_y):
    # Get the data from the figure
    ax= plt.gca() # get axis handle
    acoustic_data = ax.lines[0]
    optic_data = ax.lines[1]

    # Find which line was closer
    #closet x data
    k_a = acoustic_data.get_xdata()
    k_o = optic_data.get_xdata()
    diff_k_a = np.abs(k_a-clic_x)
    diff_k_o = np.abs(k_o-clic_x)
    # get the x index corresponding to the closet k in the graph for both branches
    index_a = np.argmin(diff_k_a)
    index_o = np.argmin(diff_k_o)

    # compute the distance on the y axis
    y_diff_a = np.abs(acoustic_data.get_ydata()[index_a]-clic_y)
    y_diff_o = np.abs(optic_data.get_ydata()[index_o]-clic_y)

    ac_is_closer = y_diff_a<y_diff_o
    # print a x at the postion 
    if(ac_is_closer):
        return [acoustic_data.get_xdata()[index_a], acoustic_data.get_ydata()[index_a]]
    else:
        return [optic_data.get_xdata()[index_a], optic_data.get_ydata()[index_a]]

def onclick(event):
    # clear the previous point 
    ax=plt.gca()
    if len(ax.lines)==3:
        ax.lines.pop()
    # Get the coordonate of the data closest from the click
    [x,y]=get_closest_point(event.xdata,event.ydata)
    plt.plot(x, y, '.')
    fig.canvas.draw()
    
def interactive_lattice_plot(G,K):
    '''
    :param G,K: coupling
    plot acoustic and optical phonons bands
    '''
    plt.close()
    fig=plt.figure()
    cid = fig.canvas.mpl_connect('button_press_event', onclick)

    k = np.linspace(-np.pi/2,np.pi/2,100)

    omega_o = G+K-np.sqrt((G+K)**2-4*G*K*np.sin(k)**2)
    omega_a = G+K+np.sqrt((G+K)**2-4*G*K*np.sin(k)**2)
    plt.plot(k,np.sqrt(omega_a))
    plt.plot(k,np.sqrt(omega_o))
    plt.xlabel("k")
    plt.ylabel("omega")
    plt.show()



In [7]:
interactive_plot = interactive(interactive_lattice_plot, G=(1, 6.0), K=(1, 6.0))
interactive_plot


interactive(children=(FloatSlider(value=3.5, description='G', max=6.0, min=1.0), FloatSlider(value=3.5, descri…

[-0.015866629563584755, 0.0296834344858312]